# Error handling

## Panic

* A program panics when it encounters something so messed up that there must be a bug in the program itself. Something like:
  * Out-of-bounds array access
  * Integer division by zero
  * Calling `.unwrap()` on an `Option` that happens to be `None`
  * Assertion failure
* A panic is not a crash. It’s not undefined behavior. The behavior is well-defined!
* Macro `panic!` can trigger a panic directly

In [ ]:
panic!("Critical error!");

### Unwinding

* Panic typically proceeds as follows:
  * An error message is printed to the terminal. If you set the `RUST_BACKTRACE` environment variable, as the messages suggests, Rust will also dump the stack at this point.
  * The stack is unwound. This is a lot like C++ exception handling. Any temporary values, local variables, or arguments that the current function was using are dropped, in the reverse of the order they were created. Dropping a value simply means cleaning up after it: any `String`s or `Vec`s the program was using are freed, any open `File`s are closed, and so on. User-defined `drop` methods are called too.
  * Finally, the thread exits. If the panicking thread was the main thread, then the whole process exits (with a nonzero exit code).

### Aborting

* Stack unwinding is the default panic behavior, but there are two circumstances in which Rust does not try to unwind the stack.
  * If a `.drop()` method triggers a second panic while Rust is still trying to clean up after the first, this is considered fatal. Rust stops unwinding and aborts the whole process.
  * Rust’s panic behavior is customizable. If you compile with `-C panic=abort`, the first panic in your program immediately aborts the process. 

## Result

* A `Result` represents an operation that can either succeed (returning a result) or fail (returning an error).
* It is defined as:

In [29]:
mod explain {
    pub enum Result<T, E> {
        Ok(T),
        Err(E),
    }
}

In [2]:
#[derive(Debug)]
struct ReadFileError {
    message: String
}

In [3]:
fn read_file(path: &str) -> Result<String, ReadFileError> {  
    let file = std::fs::read_to_string(path);
    
    match file {
        Ok(content) => Result::Ok(content),
        Err(err) => Result::Err(ReadFileError{message: err.to_string()}),
    }
}

In [6]:
let file_content = read_file("file.txt");

match file_content {
    Ok(content) => println!("{}", content),
    Err(err) => println!("Error: {:?}", err),
}

This is a file with some text.


()

### Result methods

* `result.unwrap()` also returns the success value, if result is a success result. However, if result is an error result, this method panics.
* `result.expect(message)` is the same as `.unwrap()`, but lets you provide a message that it prints in case of panic.
* `result.is_ok()` and `result.is_err()` return a `bool` telling if result is a success result or an error result.
* `result.ok()` returns the success value, if any, as an `Option<T>`. If result is a success result, this returns `Some(success_value)`; otherwise, it returns `None`, discarding the error value.
* `result.err()` returns the error value, if any, as an `Option<E>`.
* `result.unwrap_or(fallback)` returns the success value, if result is a success result. Otherwise, it returns `fallback`, discarding the error value.

In [7]:
static DEFAULT_FILE_CONTENT: &str = "Default content";

let file_content = read_file("unknown_file.txt").unwrap_or(DEFAULT_FILE_CONTENT.to_string());

file_content

"Default content"

* `result.unwrap_or_else(fallback_fn)` is the same, but instead of passing a fallback value directly, you pass a function or closure. This is for cases where it would be wasteful to compute a fallback value if you’re not going to use it. The `fallback_fn` is called only if we have an error result.
* `result.as_ref()` converts a `Result<T, E>` to a `Result<&T, &E>`, borrowing a reference to the success or error value in the existing result.
* `result.as_mut()` is the same, but borrows a mutable reference. The return type is `Result<&mut T, &mut E>`.

### Result Type Aliases

* Modules often define a `Result` type alias to avoid having to repeat an error type that’s used consistently by almost every function in the module

In [8]:
mod custom {
    pub type Result<T> = std::result::Result<T, ReadFileError>;

    #[derive(Debug)]
    pub struct ReadFileError {
        message: String
    }

    pub fn read_file(path: &str) -> Result<String> {  
        let file = std::fs::read_to_string(path);
    
        match file {
            Ok(content) => Result::Ok(content),
            Err(err) => Result::Err(ReadFileError{message: err.to_string()}),
        }
    }
}

custom::read_file("file.txt").ok()

Some("This is a file with some text.")

### Printing Errors

* All of standard library errors have a common interface, the `std::error::Error` trait, which means they share the following features:
  * They’re all printable using `println!()`. Printing an error with the `{}` format specifier typically displays only a brief error message. Alternatively, you can print with the `{:?}` format specifier, to get a `Debug` view of the error
  * `err.description()` returns an error message as a `&str`.
  * `err.source()` returns an `Option<&(dyn Error + 'static)>`: the underlying error, if any, that triggered `err`.

### Propagating Error

* In most places where we try something that could fail, we don’t want to catch and handle the error immediately
* Instead, if an error occurs, we usually want to let our caller deal with it. We want errors to propagate up the call stack.
* Rust has a `?` operator that does this. You can add a `?` to any expression that produces a `Result`, such as the result of a function call:

In [12]:
use std::io;

fn read_file_content(name: &str) -> io::Result<String> {
    use std::fs::File;
    use std::io::prelude::*;
 
    let mut file = File::open(name)?;     
    let mut contents = String::new();
    file.read_to_string(&mut contents)?;
    Ok(contents)
}

In [13]:
let file_content = read_file_content("unknown_file.txt");

match file_content {
    Ok(content) => println!("{}", content),
    Err(err) => println!("Error: {:?}", err),
}

Error: Os { code: 2, kind: NotFound, message: "Nie można odnaleźć określonego pliku." }


()

* The behavior of `?` depends on whether this function returns a success result or an error result:
  * on success, it unwraps the Result to get the success value inside.
  * on error, it immediately returns from the enclosing function, passing the error result up the call chain. To ensure that this works, `?` can only be used in functions that have a `Result` return type.

### Working with Multiple Error Types

In [16]:
use std::io::{self, BufRead};

fn read_numbers(file: &mut dyn BufRead) -> Result<Vec<i32>, io::Error> {
    let mut numbers = Vec::new();
    for line_result in file.lines() {
        let line = line_result?;
        let number = line.parse::<i32>()?;
        numbers.push(number);
    }
    Ok(numbers)
}

Error: `?` couldn't convert the error to `std::io::Error`

#### Case 1 - Trait `From`

In [18]:
use std::io::BufRead;

#[derive(Debug)]
enum ReadNumbersError {
    Io(io::Error),
    ParseInt(std::num::ParseIntError),
}

impl From<io::Error> for ReadNumbersError {
    fn from(err: io::Error) -> Self {
        ReadNumbersError::Io(err)
    }
}

impl From<std::num::ParseIntError> for ReadNumbersError {
    fn from(err: std::num::ParseIntError) -> Self {
        ReadNumbersError::ParseInt(err)
    }
}

fn read_numbers(file: &mut dyn BufRead) -> Result<Vec<i32>, ReadNumbersError> {
    let mut numbers = Vec::new();
    for line_result in file.lines() {
        let line = line_result?;
        let number = line.parse::<i32>()?;
        numbers.push(number);
    }
    Ok(numbers)
}

In [24]:
use std::io::{self, BufReader};

fn main() {
    let file = std::fs::File::open("numbers.txt").unwrap();

    let mut buffer_reader = BufReader::new(file);

    let numbers_result = read_numbers(&mut buffer_reader);

    match numbers_result {
        Ok(numbers) => println!("{:?}", numbers),
        Err(e) => println!("Error reading numbers: {:?}", e),
    }
}

main();

[1, 2, 3, 42, 665, 6, 7, 8]


#### Case 2 - `Box<dyn std::error::Error>`

* Code does not compile - there is no conversion from `ParseIntError` to `io::Error`
* The easiest way to handle multiple error types is to define these type aliases:

In [26]:
type GenericError = Box<dyn std::error::Error>;
type GenericResult<T> = Result<T, GenericError>;

* and change return type from function to `GenericResult<Vec<i32>>`

In [27]:
use std::io::{self, BufRead};

fn read_numbers(file: &mut dyn BufRead) -> GenericResult<Vec<i32>> {
    let mut numbers = Vec::new();
    for line_result in file.lines() {
        let line = line_result?;
        let number = line.parse::<i32>()?;
        numbers.push(number);
    }
    Ok(numbers)
}

In [33]:
use std::io::{self, BufReader};

fn main() {
    let file = std::fs::File::open("invalid_numbers.txt").unwrap();
    let mut buffer_reader = BufReader::new(file);
    
    let numbers_result = read_numbers(&mut buffer_reader);

    match numbers_result {
        Ok(numbers) => println!("{:?}", numbers),
        Err(e) => println!("Error reading numbers: {:?}", e),
    }
}
main();

Error reading numbers: ParseIntError { kind: InvalidDigit }


### Declaring a Custom Error Types

In [32]:
#[derive(Debug, Clone)]
pub struct JsonError {
    pub message: String,
    pub line: usize,
    pub column: usize,
}

In [34]:
use std;
use std::fmt;

// Errors should be printable.
impl fmt::Display for JsonError {
    fn fmt(&self, f: &mut fmt::Formatter) -> Result<(), fmt::Error> {
        write!(f, "{} ({}:{})", self.message, self.line, self.column)
    }
}

// Errors should implement the std::error::Error trait.
impl std::error::Error for JsonError {
    fn description(&self) -> &str {
        &self.message
    }
}

In [40]:
:dep serde_json = "1.0.64"
:dep serde = { version="1.0.127", features=["derive"] }

use serde::{Deserialize, Serialize};

#[derive(Serialize, Deserialize, Debug)]
struct Person {
    name: String,
    age: i32,
}

fn load_from_json() -> Result<Person, JsonError> {
    let json = r#"
        {
            "name": "John",
            "age": 30
        }
    "#;

    let person: Person = match serde_json::from_str(json) {
        Ok(data) => data,
        Err(err) => {
            let line = err.line();
            let column = err.column();
            let message = err.to_string();
            return Err(JsonError { message, line, column });
        }
    };

    Ok(person)
}

In [41]:
let person = load_from_json().err();
person

None